# Lab 1: MLP 影像分類 (CIFAR-10)

本實驗將使用 **多層感知機 (Multi-Layer Perceptron, MLP)** 對 CIFAR-10 手寫數字資料集進行分類。

## 學習目標
- 理解 Tensor 展平 (Flattening) 的概念
- 實作全連接層 (Linear Layers)
- 學習 SGD 優化器的基礎應用
- 熟悉訓練與驗證流程

## 1. 環境設定與套件匯入

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# 匯入自定義模組
from dataset import CIFAR-10Dataset
from model import create_mlp_model
from train import Trainer
from eval import (
    evaluate_model,
    plot_confusion_matrix,
    print_classification_report,
    visualize_predictions,
    visualize_misclassified
)
from utils import set_seed, get_device, plot_training_curves, save_metrics

# 設定中文字體 (避免圖表中文顯示問題)
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print('✅ 套件匯入成功!')

## 2. 設定超參數與隨機種子

In [ ]:
# 超參數設定
CONFIG = {
    'seed': 42,
    'batch_size': 128,
    'num_epochs: 50,
    'learning_rate': 0.01,
    'base_channels': [64],
    'dropout_rate': 0.3,
    'num_workers': 2,
    'save_dir': './checkpoints'
}

# 設定隨機種子以確保實驗可重現
set_seed(CONFIG['seed'])

# 取得運算裝置
device = get_device()

print(f'\n📋 超參數設定:')
for key, value in CONFIG.items():
    print(f'   {key}: {value}')

## 3. 載入資料集

In [ ]:
# 建立資料集
dataset = CIFAR-10Dataset(
    data_dir='./data',
    batch_size=CONFIG['batch_size'],
    num_workers=CONFIG['num_workers']
)

# 取得 DataLoader
train_loader = dataset.get_train_loader()
test_loader = dataset.get_test_loader()
class_names = dataset.get_class_names()

print(f'\n✅ DataLoader 建立完成!')
print(f'   訓練批次數: {len(train_loader)}')
print(f'   測試批次數: {len(test_loader)}')

### 視覺化資料樣本

In [ ]:
# 顯示一些樣本
images, labels = dataset.get_sample_batch(num_samples=16)

fig, axes = plt.subplots(4, 4, figsize=(10, 10))
axes = axes.flatten()

for idx in range(16):
    ax = axes[idx]
    img = images[idx].squeeze()
    
    # 反正規化以便顯示
    mean, std = 0.1307, 0.3081
    img = img * std + mean
    img = torch.clamp(img, 0, 1)
    
    ax.imshow(img, cmap='gray')
    ax.set_title(f'標籤: {labels[idx]}', fontsize=12, fontweight='bold')
    ax.axis('off')

plt.suptitle('CIFAR-10 資料集樣本', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. 建立模型

In [ ]:
# 建立 MLP 模型
model = create_mlp_model(
    input_size=784,  # 28 * 28
    base_channels=CONFIG['hidden_sizes'],
    num_classes=10,
    dropout_rate=CONFIG['dropout_rate']
)

# 將模型移至指定裝置
model = model.to(device)

print(f'\n🏗️  模型架構:')
print(model)

## 5. 定義損失函數與優化器

In [ ]:
# 損失函數 (CrossEntropyLoss 已包含 Softmax)
criterion = nn.CrossEntropyLoss()

# 優化器 (SGD with Momentum)
optimizer = optim.SGD(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    momentum=0.9,
    weight_decay=1e-4
)

# 學習率調度器 (每 7 個 epoch 減少學習率)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print('✅ 損失函數與優化器設定完成!')
print(f'   損失函數: {criterion.__class__.__name__}')
print(f'   優化器: {optimizer.__class__.__name__}')
print(f'   初始學習率: {CONFIG["learning_rate"]}')

## 6. 訓練模型

In [ ]:
# 建立 Trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,  # 使用測試集作為驗證集
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    save_dir=CONFIG['save_dir'],
    scheduler=scheduler
)

# 開始訓練
history = trainer.train(num_epochs=CONFIG['num_epochs'])

## 7. 視覺化訓練曲線

In [ ]:
# 繪製訓練曲線
plot_training_curves(
    train_losses=history['train_losses'],
    val_losses=history['val_losses'],
    train_accs=history['train_accs'],
    val_accs=history['val_accs'],
    save_path='./results/training_curves.png'
)

## 8. 評估模型

In [ ]:
# 評估模型
test_loss, test_acc, predictions, labels = evaluate_model(
    model=model,
    data_loader=test_loader,
    device=device,
    criterion=criterion
)

print(f'\n📊 測試集評估結果:')
print(f'   測試損失: {test_loss:.4f}')
print(f'   測試準確率: {test_acc:.2f}%')

### 繪製混淆矩陣

In [ ]:
plot_confusion_matrix(
    y_true=labels,
    y_pred=predictions,
    class_names=class_names,
    save_path='./results/confusion_matrix.png'
)

### 輸出分類報告

In [ ]:
print_classification_report(
    y_true=labels,
    y_pred=predictions,
    class_names=class_names
)

## 9. 視覺化預測結果

In [ ]:
# 視覺化預測結果
visualize_predictions(
    model=model,
    data_loader=test_loader,
    device=device,
    class_names=class_names,
    num_samples=16,
    save_path='./results/predictions.png'
)

### 視覺化預測錯誤的樣本

In [ ]:
# 視覺化預測錯誤的樣本
visualize_misclassified(
    model=model,
    data_loader=test_loader,
    device=device,
    class_names=class_names,
    num_samples=16,
    save_path='./results/misclassified.png'
)

## 10. 儲存訓練指標

In [ ]:
# 儲存訓練指標至 JSON
metrics = {
    'config': CONFIG,
    'history': {
        'train_losses': history['train_losses'],
        'val_losses': history['val_losses'],
        'train_accs': history['train_accs'],
        'val_accs': history['val_accs']
    },
    'final_results': {
        'best_val_acc': history['best_val_acc'],
        'best_epoch': history['best_epoch'],
        'test_loss': test_loss,
        'test_acc': test_acc
    }
}

save_metrics(metrics, './results/metrics.json')

print('\n🎉 訓練與評估完成!')

## 總結

在本實驗中，你已經:
1. ✅ 載入並預處理 CIFAR-10 資料集
2. ✅ 建立多層感知機 (MLP) 模型
3. ✅ 實作訓練與驗證流程
4. ✅ 評估模型表現並視覺化結果

### 思考問題
1. 為什麼需要將影像展平 (Flatten) 才能輸入 MLP?
2. Dropout 的作用是什麼? 如何幫助模型泛化?
3. 觀察混淆矩陣，哪些數字最容易被混淆?
4. 如何進一步提升模型的準確率?